#  adaBoost

In [2]:
import pandas as pd

data_path = './data/'


train_data = pd.read_csv(f'{data_path}train_data.csv')
test_data = pd.read_csv(f'{data_path}test_data.csv')

# Selección de características con mrMr
df_reduce_mrmr = pd.read_csv(f'{data_path}X_train_reduce_mrmr.csv')
df_reduce_mrmr_instances = pd.read_csv(f'{data_path}df_reduce_mrmr_instances.csv')
df_reduce_mrmr_instances_hard = pd.read_csv(f'{data_path}df_reduce_mrmr_instances_hard.csv')
df_reduce_mrmr_instances_GLVQ = pd.read_csv(f'{data_path}df_reduce_mrmr_instances_GLVQ.csv')

# Selección de características con RFC
df_X_train_reduce_RFC = pd.read_csv(f'{data_path}df_X_train_reduce_RFC.csv')
df_reduce_RFC_instances = pd.read_csv(f'{data_path}df_reduce_RFC_instances.csv')
df_reduce_RFC_instances_hard = pd.read_csv(f'{data_path}df_reduce_RFC_instances_hard.csv')
df_reduce_RFC_instances_GLVQ = pd.read_csv(f'{data_path}df_reduce_RFC_instances_GLVQ.csv')

# Mostrar información sobre los DataFrames cargados
print("Datos cargados exitosamente:")
print(f"train_data: {train_data.shape}")
print(f"df_reduce_mrmr: {df_reduce_mrmr.shape}")
print(f"df_reduce_mrmr_instances: {df_reduce_mrmr_instances.shape}")
print(f"df_reduce_mrmr_instances_GLVQ: {df_reduce_mrmr_instances_GLVQ.shape}")
print(f"df_X_train_reduce_RFC: {df_X_train_reduce_RFC.shape}")
print(f"df_reduce_RFC_instances: {df_reduce_RFC_instances.shape}")
print(f"df_reduce_RFC_instances_GLVQ: {df_reduce_RFC_instances_GLVQ.shape}")


Datos cargados exitosamente:
train_data: (256326, 31)
df_reduce_mrmr: (256326, 11)
df_reduce_mrmr_instances: (886, 11)
df_reduce_mrmr_instances_GLVQ: (2, 11)
df_X_train_reduce_RFC: (256326, 11)
df_reduce_RFC_instances: (886, 11)
df_reduce_RFC_instances_GLVQ: (2, 11)


In [20]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# adaBoost con mrmr y ClusterCentroids soft

## DecisionTreeClassifier

In [8]:
X = df_reduce_mrmr_instances.drop(columns=['Class'])
y = df_reduce_mrmr_instances['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("Dimensiones de los conjuntos:")
print(f"Conjunto de entrenamiento: {X_train.shape}, {y_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}, {y_test.shape}")

base_estimator = DecisionTreeClassifier(max_depth=1)  # Clasificador predeterminado
adaboost = AdaBoostClassifier(estimator=base_estimator, random_state=0)


param_grid = {
    'n_estimators': [10, 50, 100],      # Número de clasificadores débiles
    'learning_rate': [0.1, 0.5, 1.0],  # Tasa de aprendizaje
    'estimator__max_depth': [1, 2, 3]  # Profundidad del árbol base
}

# Creamos el GridSearchCV
grid_search = GridSearchCV(estimator=adaboost, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Mostramos los mejores parámetros y el desempeño
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

# Evaluamos el mejor modelo en los datos de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Informe completo en datos de prueba\n", classification_report(y_test, y_pred))

Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Mejores parámetros: {'estimator__max_depth': 2, 'learning_rate': 1.0, 'n_estimators': 50}
Mejor puntuación de validación cruzada: 0.9562481270602337
Informe completo en datos de prueba
               precision    recall  f1-score   support

           0       0.97      0.99      0.98        89
           1       0.99      0.97      0.98        89

    accuracy                           0.98       178
   macro avg       0.98      0.98      0.98       178
weighted avg       0.98      0.98      0.98       178



### Aplicamos adaBoost con los mejores parámetros en el conjunto de test

In [9]:
from sklearn.preprocessing import MinMaxScaler
# Cogemos los datos de test y les eliminamos las cracterísticas que no necesitamos
X = test_data.drop(columns=['Class'])
y_test_final = test_data['Class']
columns_to_keep_mrmr = ['V17', 'Time', 'Amount', 'V25', 'V20', 'V7', 'V13', 'V22', 'V19', 'V23']
X_reduce = X[columns_to_keep_mrmr]

#normalizamos la entrada
scaler = MinMaxScaler()
# Normalizar las columnas
X_reduce['Amount'] = scaler.fit_transform(X_reduce[['Amount']])
X_reduce['Time'] = scaler.fit_transform(X_reduce[['Time']])

print(y_test_final.value_counts())

Class
0    28432
1       49
Name: count, dtype: int64


C:\Users\maris\AppData\Local\Temp\ipykernel_9080\3588149006.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_reduce['Amount'] = scaler.fit_transform(X_reduce[['Amount']])
C:\Users\maris\AppData\Local\Temp\ipykernel_9080\3588149006.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_reduce['Time'] = scaler.fit_transform(X_reduce[['Time']])


In [13]:
# Configura el modelo con los mejores parámetros
base_estimator = DecisionTreeClassifier(max_depth=2) 
adaboost = AdaBoostClassifier(
    estimator=base_estimator,
    n_estimators=50,         
    learning_rate=1.0,       
    random_state=0           
)


X = df_reduce_mrmr_instances.drop(columns=['Class'])
y = df_reduce_mrmr_instances['Class']

# Entrenamos el modelo con los datos de entrenamiento
adaboost.fit(X, y)

# Evalúa el modelo en los datos de prueba
y_pred_tree = adaboost.predict(X_reduce)

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test_final, y_pred_tree)
report = classification_report(y_test_final, y_pred_tree, target_names=['Correctas', 'Fraudulentas'])
# Mostrar la matriz de confusión
print("Matriz de confusión:")
print(conf_matrix)

print("Reporte de Clasificación:")
print(report)

Matriz de confusión:
[[14622 13810]
 [    2    47]]
Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.51      0.68     28432
Fraudulentas       0.00      0.96      0.01        49

    accuracy                           0.52     28481
   macro avg       0.50      0.74      0.34     28481
weighted avg       1.00      0.52      0.68     28481



Probamos otros clasificadores débiles como LogisticRegression o SVM.

## LogisticRegression

In [14]:
X = df_reduce_mrmr_instances.drop(columns=['Class'])
y = df_reduce_mrmr_instances['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("Dimensiones de los conjuntos:")
print(f"Conjunto de entrenamiento: {X_train.shape}, {y_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}, {y_test.shape}")

base_estimator=LogisticRegression(max_iter=1000)  
adaboost = AdaBoostClassifier(estimator=base_estimator, random_state=0)


param_grid = {
    'n_estimators': [10, 50, 100],      # Número de clasificadores débiles
    'learning_rate': [0.1, 0.5, 1.0],  # Tasa de aprendizaje
}

# Creamos el GridSearchCV
grid_search = GridSearchCV(estimator=adaboost, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Mostramos los mejores parámetros y el desempeño
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

# Evaluamos el mejor modelo en los datos de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Informe completo en datos de prueba\n", classification_report(y_test, y_pred))

Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Mejores parámetros: {'learning_rate': 0.1, 'n_estimators': 50}
Mejor puntuación de validación cruzada: 0.8390070921985815
Informe completo en datos de prueba
               precision    recall  f1-score   support

           0       0.80      0.98      0.88        89
           1       0.97      0.75      0.85        89

    accuracy                           0.87       178
   macro avg       0.88      0.87      0.86       178
weighted avg       0.88      0.87      0.86       178



### Aplicamos adaBoost con los mejores parámetros en el conjunto de test

In [15]:
# Configura el modelo con los mejores parámetros
base_estimator = LogisticRegression(max_iter=1000)   
adaboost = AdaBoostClassifier(
    estimator=base_estimator,
    n_estimators=50,         
    learning_rate=0.1,       
    random_state=0           
)


X = df_reduce_mrmr_instances.drop(columns=['Class'])
y = df_reduce_mrmr_instances['Class']

# Entrenamos el modelo con los datos de entrenamiento
adaboost.fit(X, y)

# Evalúa el modelo en los datos de prueba
y_pred_lr = adaboost.predict(X_reduce)

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test_final, y_pred_lr)
report = classification_report(y_test_final, y_pred_lr, target_names=['Correctas', 'Fraudulentas'])
# Mostrar la matriz de confusión
print("Matriz de confusión:")
print(conf_matrix)

print("Reporte de Clasificación:")
print(report)

Matriz de confusión:
[[26546  1886]
 [   11    38]]
Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.93      0.97     28432
Fraudulentas       0.02      0.78      0.04        49

    accuracy                           0.93     28481
   macro avg       0.51      0.85      0.50     28481
weighted avg       1.00      0.93      0.96     28481



## SVM

In [19]:
X = df_reduce_mrmr_instances.drop(columns=['Class'])
y = df_reduce_mrmr_instances['Class']
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("Dimensiones de los conjuntos:")
print(f"Conjunto de entrenamiento: {X_train.shape}, {y_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}, {y_test.shape}")

base_estimator = SVC(kernel='linear', random_state=0) 
adaboost = AdaBoostClassifier(estimator=base_estimator, random_state=0)


param_grid = {
    'n_estimators': [10, 50, 100],      # Número de clasificadores débiles
    'learning_rate': [0.1, 0.5, 1.0],  # Tasa de aprendizaje
    'estimator__C': [0.1, 1, 10]
}

# Creamos el GridSearchCV
grid_search = GridSearchCV(estimator=adaboost, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Mostramos los mejores parámetros y el desempeño
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

# Evaluamos el mejor modelo en los datos de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Informe completo en datos de prueba\n", classification_report(y_test, y_pred))

Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Mejores parámetros: {'estimator__C': 10, 'learning_rate': 0.1, 'n_estimators': 10}
Mejor puntuación de validación cruzada: 0.8460493457197084
Informe completo en datos de prueba
               precision    recall  f1-score   support

           0       0.77      1.00      0.87        89
           1       1.00      0.71      0.83        89

    accuracy                           0.85       178
   macro avg       0.89      0.85      0.85       178
weighted avg       0.89      0.85      0.85       178



### Aplicamos adaBoost con los mejores parámetros en el conjunto de test

In [21]:
# Configura el modelo con los mejores parámetros
base_estimator = SVC(kernel='linear', random_state=0, C=10)   
adaboost = AdaBoostClassifier(
    estimator=base_estimator,
    n_estimators=10,         
    learning_rate=0.1,       
    random_state=0           
)


X = df_reduce_mrmr_instances.drop(columns=['Class'])
y = df_reduce_mrmr_instances['Class']

# Entrenamos el modelo con los datos de entrenamiento
adaboost.fit(X, y)

# Evalúa el modelo en los datos de prueba
y_pred_lr = adaboost.predict(X_reduce)

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test_final, y_pred_lr)
report = classification_report(y_test_final, y_pred_lr, target_names=['Correctas', 'Fraudulentas'])
# Mostrar la matriz de confusión
print("Matriz de confusión:")
print(conf_matrix)

print("Reporte de Clasificación:")
print(report)

Matriz de confusión:
[[28365    67]
 [   14    35]]
Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      1.00      1.00     28432
Fraudulentas       0.34      0.71      0.46        49

    accuracy                           1.00     28481
   macro avg       0.67      0.86      0.73     28481
weighted avg       1.00      1.00      1.00     28481



# adaBoost con mrmr y ClusterCentroids hard

## DecisionTreeClassifier

In [22]:
X = df_reduce_mrmr_instances_hard.drop(columns=['Class'])
y = df_reduce_mrmr_instances_hard['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("Dimensiones de los conjuntos:")
print(f"Conjunto de entrenamiento: {X_train.shape}, {y_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}, {y_test.shape}")

base_estimator = DecisionTreeClassifier(max_depth=1)  # Clasificador predeterminado
adaboost = AdaBoostClassifier(estimator=base_estimator, random_state=0)


param_grid = {
    'n_estimators': [10, 50, 100],      # Número de clasificadores débiles
    'learning_rate': [0.1, 0.5, 1.0],  # Tasa de aprendizaje
    'estimator__max_depth': [1, 2, 3]  # Profundidad del árbol base
}

# Creamos el GridSearchCV
grid_search = GridSearchCV(estimator=adaboost, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Mostramos los mejores parámetros y el desempeño
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

# Evaluamos el mejor modelo en los datos de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Informe completo en datos de prueba\n", classification_report(y_test, y_pred))

Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Mejores parámetros: {'estimator__max_depth': 3, 'learning_rate': 0.5, 'n_estimators': 100}
Mejor puntuación de validación cruzada: 0.9139047048246928
Informe completo en datos de prueba
               precision    recall  f1-score   support

           0       0.87      0.93      0.90        89
           1       0.93      0.87      0.90        89

    accuracy                           0.90       178
   macro avg       0.90      0.90      0.90       178
weighted avg       0.90      0.90      0.90       178



### Aplicamos adaBoost con los mejores parámetros en el conjunto de test

In [23]:
# Configura el modelo con los mejores parámetros
base_estimator = DecisionTreeClassifier(max_depth=3) 
adaboost = AdaBoostClassifier(
    estimator=base_estimator,
    n_estimators=100,         
    learning_rate=0.5,       
    random_state=0           
)


X = df_reduce_mrmr_instances_hard.drop(columns=['Class'])
y = df_reduce_mrmr_instances_hard['Class']

# Entrenamos el modelo con los datos de entrenamiento
adaboost.fit(X, y)

# Evalúa el modelo en los datos de prueba
y_pred_tree = adaboost.predict(X_reduce)

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test_final, y_pred_tree)
report = classification_report(y_test_final, y_pred_tree, target_names=['Correctas', 'Fraudulentas'])
# Mostrar la matriz de confusión
print("Matriz de confusión:")
print(conf_matrix)

print("Reporte de Clasificación:")
print(report)

Matriz de confusión:
[[26333  2099]
 [    5    44]]
Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.93      0.96     28432
Fraudulentas       0.02      0.90      0.04        49

    accuracy                           0.93     28481
   macro avg       0.51      0.91      0.50     28481
weighted avg       1.00      0.93      0.96     28481



Probamos otros clasificadores débiles como LogisticRegression o SVM.

## LogisticRegression

In [24]:
X = df_reduce_mrmr_instances_hard.drop(columns=['Class'])
y = df_reduce_mrmr_instances_hard['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("Dimensiones de los conjuntos:")
print(f"Conjunto de entrenamiento: {X_train.shape}, {y_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}, {y_test.shape}")

base_estimator=LogisticRegression(max_iter=1000)  
adaboost = AdaBoostClassifier(estimator=base_estimator, random_state=0)


param_grid = {
    'n_estimators': [10, 50, 100],      # Número de clasificadores débiles
    'learning_rate': [0.1, 0.5, 1.0],  # Tasa de aprendizaje
}

# Creamos el GridSearchCV
grid_search = GridSearchCV(estimator=adaboost, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Mostramos los mejores parámetros y el desempeño
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

# Evaluamos el mejor modelo en los datos de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Informe completo en datos de prueba\n", classification_report(y_test, y_pred))

Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Mejores parámetros: {'learning_rate': 0.1, 'n_estimators': 50}
Mejor puntuación de validación cruzada: 0.8418439716312058
Informe completo en datos de prueba
               precision    recall  f1-score   support

           0       0.79      1.00      0.89        89
           1       1.00      0.74      0.85        89

    accuracy                           0.87       178
   macro avg       0.90      0.87      0.87       178
weighted avg       0.90      0.87      0.87       178



### Aplicamos adaBoost con los mejores parámetros en el conjunto de test

In [25]:
# Configura el modelo con los mejores parámetros
base_estimator = LogisticRegression(max_iter=1000)   
adaboost = AdaBoostClassifier(
    estimator=base_estimator,
    n_estimators=50,         
    learning_rate=0.1,       
    random_state=0           
)


X = df_reduce_mrmr_instances_hard.drop(columns=['Class'])
y = df_reduce_mrmr_instances_hard['Class']

# Entrenamos el modelo con los datos de entrenamiento
adaboost.fit(X, y)

# Evalúa el modelo en los datos de prueba
y_pred_lr = adaboost.predict(X_reduce)

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test_final, y_pred_lr)
report = classification_report(y_test_final, y_pred_lr, target_names=['Correctas', 'Fraudulentas'])
# Mostrar la matriz de confusión
print("Matriz de confusión:")
print(conf_matrix)

print("Reporte de Clasificación:")
print(report)

Matriz de confusión:
[[26606  1826]
 [   11    38]]
Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.94      0.97     28432
Fraudulentas       0.02      0.78      0.04        49

    accuracy                           0.94     28481
   macro avg       0.51      0.86      0.50     28481
weighted avg       1.00      0.94      0.97     28481



## SVM

In [26]:
X = df_reduce_mrmr_instances_hard.drop(columns=['Class'])
y = df_reduce_mrmr_instances_hard['Class']
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("Dimensiones de los conjuntos:")
print(f"Conjunto de entrenamiento: {X_train.shape}, {y_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}, {y_test.shape}")

base_estimator = SVC(kernel='linear', random_state=0) 
adaboost = AdaBoostClassifier(estimator=base_estimator, random_state=0)


param_grid = {
    'n_estimators': [10, 50, 100],      # Número de clasificadores débiles
    'learning_rate': [0.1, 0.5, 1.0],  # Tasa de aprendizaje
    'estimator__C': [0.1, 1, 10]
}

# Creamos el GridSearchCV
grid_search = GridSearchCV(estimator=adaboost, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Mostramos los mejores parámetros y el desempeño
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

# Evaluamos el mejor modelo en los datos de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Informe completo en datos de prueba\n", classification_report(y_test, y_pred))

Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Mejores parámetros: {'estimator__C': 10, 'learning_rate': 0.1, 'n_estimators': 10}
Mejor puntuación de validación cruzada: 0.8432324443112575
Informe completo en datos de prueba
               precision    recall  f1-score   support

           0       0.77      1.00      0.87        89
           1       1.00      0.71      0.83        89

    accuracy                           0.85       178
   macro avg       0.89      0.85      0.85       178
weighted avg       0.89      0.85      0.85       178



### Aplicamos adaBoost con los mejores parámetros en el conjunto de test

In [28]:
# Configura el modelo con los mejores parámetros
base_estimator = SVC(kernel='linear', random_state=0, C=10)   
adaboost = AdaBoostClassifier(
    estimator=base_estimator,
    n_estimators=10,         
    learning_rate=0.1,       
    random_state=0           
)


X = df_reduce_mrmr_instances_hard.drop(columns=['Class'])
y = df_reduce_mrmr_instances_hard['Class']

# Entrenamos el modelo con los datos de entrenamiento
adaboost.fit(X, y)

# Evalúa el modelo en los datos de prueba
y_pred_lr = adaboost.predict(X_reduce)

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test_final, y_pred_lr)
report = classification_report(y_test_final, y_pred_lr, target_names=['Correctas', 'Fraudulentas'])
# Mostrar la matriz de confusión
print("Matriz de confusión:")
print(conf_matrix)

print("Reporte de Clasificación:")
print(report)

Matriz de confusión:
[[28384    48]
 [   14    35]]
Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      1.00      1.00     28432
Fraudulentas       0.42      0.71      0.53        49

    accuracy                           1.00     28481
   macro avg       0.71      0.86      0.76     28481
weighted avg       1.00      1.00      1.00     28481



# adaBoost con RFC y ClusterCentroids soft

## DecisionTreeClassifier

In [30]:
X = df_reduce_RFC_instances.drop(columns=['Class'])
y = df_reduce_RFC_instances['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("Dimensiones de los conjuntos:")
print(f"Conjunto de entrenamiento: {X_train.shape}, {y_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}, {y_test.shape}")

base_estimator = DecisionTreeClassifier(max_depth=1)  # Clasificador predeterminado
adaboost = AdaBoostClassifier(estimator=base_estimator, random_state=0)


param_grid = {
    'n_estimators': [10, 50, 100],      # Número de clasificadores débiles
    'learning_rate': [0.1, 0.5, 1.0],  # Tasa de aprendizaje
    'estimator__max_depth': [1, 2, 3]  # Profundidad del árbol base
}

# Creamos el GridSearchCV
grid_search = GridSearchCV(estimator=adaboost, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Mostramos los mejores parámetros y el desempeño
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

# Evaluamos el mejor modelo en los datos de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Informe completo en datos de prueba\n", classification_report(y_test, y_pred))

Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Mejores parámetros: {'estimator__max_depth': 2, 'learning_rate': 0.5, 'n_estimators': 50}
Mejor puntuación de validación cruzada: 0.9279792228548598
Informe completo en datos de prueba
               precision    recall  f1-score   support

           0       0.88      0.96      0.91        89
           1       0.95      0.87      0.91        89

    accuracy                           0.91       178
   macro avg       0.91      0.91      0.91       178
weighted avg       0.91      0.91      0.91       178



### Aplicamos adaBoost con los mejores parámetros en el conjunto de test

In [32]:
# Cogemos los datos de test y les eliminamos las cracterísticas que no necesitamos
X = test_data.drop(columns=['Class'])
y_test_final = test_data['Class']
columns_to_keep_RFC = ['V17', 'V16', 'V12', 'V14', 'V11', 'V10', 'V9', 'V4', 'V18', 'V7']
X_reduce_RFC = X[columns_to_keep_RFC]


print(y_test_final.value_counts())

Class
0    28432
1       49
Name: count, dtype: int64


In [34]:
# Configura el modelo con los mejores parámetros
base_estimator = DecisionTreeClassifier(max_depth=2) 
adaboost = AdaBoostClassifier(
    estimator=base_estimator,
    n_estimators=50,         
    learning_rate=0.5,       
    random_state=0           
)


X = df_reduce_RFC_instances.drop(columns=['Class'])
y = df_reduce_RFC_instances['Class']

# Entrenamos el modelo con los datos de entrenamiento
adaboost.fit(X, y)

# Evalúa el modelo en los datos de prueba
y_pred_tree = adaboost.predict(X_reduce_RFC)

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test_final, y_pred_tree)
report = classification_report(y_test_final, y_pred_tree, target_names=['Correctas', 'Fraudulentas'])
# Mostrar la matriz de confusión
print("Matriz de confusión:")
print(conf_matrix)

print("Reporte de Clasificación:")
print(report)

Matriz de confusión:
[[27664   768]
 [    6    43]]
Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.97      0.99     28432
Fraudulentas       0.05      0.88      0.10        49

    accuracy                           0.97     28481
   macro avg       0.53      0.93      0.54     28481
weighted avg       1.00      0.97      0.98     28481



Probamos otros clasificadores débiles como LogisticRegression o SVM.

## LogisticRegression

In [35]:
X = df_reduce_RFC_instances.drop(columns=['Class'])
y = df_reduce_RFC_instances['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("Dimensiones de los conjuntos:")
print(f"Conjunto de entrenamiento: {X_train.shape}, {y_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}, {y_test.shape}")

base_estimator=LogisticRegression(max_iter=1000)  
adaboost = AdaBoostClassifier(estimator=base_estimator, random_state=0)


param_grid = {
    'n_estimators': [10, 50, 100],      # Número de clasificadores débiles
    'learning_rate': [0.1, 0.5, 1.0],  # Tasa de aprendizaje
}

# Creamos el GridSearchCV
grid_search = GridSearchCV(estimator=adaboost, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Mostramos los mejores parámetros y el desempeño
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

# Evaluamos el mejor modelo en los datos de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Informe completo en datos de prueba\n", classification_report(y_test, y_pred))

Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Mejores parámetros: {'learning_rate': 0.5, 'n_estimators': 10}
Mejor puntuación de validación cruzada: 0.9251523324343223
Informe completo en datos de prueba
               precision    recall  f1-score   support

           0       0.87      0.96      0.91        89
           1       0.95      0.85      0.90        89

    accuracy                           0.90       178
   macro avg       0.91      0.90      0.90       178
weighted avg       0.91      0.90      0.90       178



### Aplicamos adaBoost con los mejores parámetros en el conjunto de test

In [36]:
# Configura el modelo con los mejores parámetros
base_estimator = LogisticRegression(max_iter=1000)   
adaboost = AdaBoostClassifier(
    estimator=base_estimator,
    n_estimators=10,         
    learning_rate=0.5,       
    random_state=0           
)


X = df_reduce_RFC_instances.drop(columns=['Class'])
y = df_reduce_RFC_instances['Class']

# Entrenamos el modelo con los datos de entrenamiento
adaboost.fit(X, y)

# Evalúa el modelo en los datos de prueba
y_pred_lr = adaboost.predict(X_reduce_RFC)

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test_final, y_pred_lr)
report = classification_report(y_test_final, y_pred_lr, target_names=['Correctas', 'Fraudulentas'])
# Mostrar la matriz de confusión
print("Matriz de confusión:")
print(conf_matrix)

print("Reporte de Clasificación:")
print(report)

Matriz de confusión:
[[28068   364]
 [    4    45]]
Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.99      0.99     28432
Fraudulentas       0.11      0.92      0.20        49

    accuracy                           0.99     28481
   macro avg       0.55      0.95      0.59     28481
weighted avg       1.00      0.99      0.99     28481



## SVM

In [37]:
X = df_reduce_RFC_instances.drop(columns=['Class'])
y = df_reduce_RFC_instances['Class']
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("Dimensiones de los conjuntos:")
print(f"Conjunto de entrenamiento: {X_train.shape}, {y_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}, {y_test.shape}")

base_estimator = SVC(kernel='linear', random_state=0) 
adaboost = AdaBoostClassifier(estimator=base_estimator, random_state=0)


param_grid = {
    'n_estimators': [10, 50, 100],      # Número de clasificadores débiles
    'learning_rate': [0.1, 0.5, 1.0],  # Tasa de aprendizaje
    'estimator__C': [0.1, 1, 10]
}

# Creamos el GridSearchCV
grid_search = GridSearchCV(estimator=adaboost, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Mostramos los mejores parámetros y el desempeño
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

# Evaluamos el mejor modelo en los datos de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Informe completo en datos de prueba\n", classification_report(y_test, y_pred))

Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Mejores parámetros: {'estimator__C': 0.1, 'learning_rate': 0.1, 'n_estimators': 50}
Mejor puntuación de validación cruzada: 0.9223354310258716
Informe completo en datos de prueba
               precision    recall  f1-score   support

           0       0.86      0.96      0.90        89
           1       0.95      0.84      0.89        89

    accuracy                           0.90       178
   macro avg       0.90      0.90      0.90       178
weighted avg       0.90      0.90      0.90       178



### Aplicamos adaBoost con los mejores parámetros en el conjunto de test

In [38]:
# Configura el modelo con los mejores parámetros
base_estimator = SVC(kernel='linear', random_state=0, C=0.1)   
adaboost = AdaBoostClassifier(
    estimator=base_estimator,
    n_estimators=50,         
    learning_rate=0.1,       
    random_state=0           
)


X = df_reduce_RFC_instances.drop(columns=['Class'])
y = df_reduce_RFC_instances['Class']

# Entrenamos el modelo con los datos de entrenamiento
adaboost.fit(X, y)

# Evalúa el modelo en los datos de prueba
y_pred_lr = adaboost.predict(X_reduce_RFC)

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test_final, y_pred_lr)
report = classification_report(y_test_final, y_pred_lr, target_names=['Correctas', 'Fraudulentas'])
# Mostrar la matriz de confusión
print("Matriz de confusión:")
print(conf_matrix)

print("Reporte de Clasificación:")
print(report)

Matriz de confusión:
[[28198   234]
 [    5    44]]
Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.99      1.00     28432
Fraudulentas       0.16      0.90      0.27        49

    accuracy                           0.99     28481
   macro avg       0.58      0.94      0.63     28481
weighted avg       1.00      0.99      0.99     28481



# adaBoost con RFC y ClusterCentroids hard

## DecisionTreeClassifier

In [39]:
X = df_reduce_RFC_instances_hard.drop(columns=['Class'])
y = df_reduce_RFC_instances_hard['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("Dimensiones de los conjuntos:")
print(f"Conjunto de entrenamiento: {X_train.shape}, {y_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}, {y_test.shape}")

base_estimator = DecisionTreeClassifier(max_depth=1)  # Clasificador predeterminado
adaboost = AdaBoostClassifier(estimator=base_estimator, random_state=0)


param_grid = {
    'n_estimators': [10, 50, 100],      # Número de clasificadores débiles
    'learning_rate': [0.1, 0.5, 1.0],  # Tasa de aprendizaje
    'estimator__max_depth': [1, 2, 3]  # Profundidad del árbol base
}

# Creamos el GridSearchCV
grid_search = GridSearchCV(estimator=adaboost, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Mostramos los mejores parámetros y el desempeño
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

# Evaluamos el mejor modelo en los datos de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Informe completo en datos de prueba\n", classification_report(y_test, y_pred))

Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Mejores parámetros: {'estimator__max_depth': 2, 'learning_rate': 0.5, 'n_estimators': 100}
Mejor puntuación de validación cruzada: 0.9308161022874838
Informe completo en datos de prueba
               precision    recall  f1-score   support

           0       0.87      0.97      0.91        89
           1       0.96      0.85      0.90        89

    accuracy                           0.91       178
   macro avg       0.92      0.91      0.91       178
weighted avg       0.92      0.91      0.91       178



### Aplicamos adaBoost con los mejores parámetros en el conjunto de test

In [41]:
# Configura el modelo con los mejores parámetros
base_estimator = DecisionTreeClassifier(max_depth=2) 
adaboost = AdaBoostClassifier(
    estimator=base_estimator,
    n_estimators=100,         
    learning_rate=0.5,       
    random_state=0           
)


X = df_reduce_RFC_instances_hard.drop(columns=['Class'])
y = df_reduce_RFC_instances_hard['Class']

# Entrenamos el modelo con los datos de entrenamiento
adaboost.fit(X, y)

# Evalúa el modelo en los datos de prueba
y_pred_tree = adaboost.predict(X_reduce_RFC)

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test_final, y_pred_tree)
report = classification_report(y_test_final, y_pred_tree, target_names=['Correctas', 'Fraudulentas'])
# Mostrar la matriz de confusión
print("Matriz de confusión:")
print(conf_matrix)

print("Reporte de Clasificación:")
print(report)

Matriz de confusión:
[[27698   734]
 [    4    45]]
Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.97      0.99     28432
Fraudulentas       0.06      0.92      0.11        49

    accuracy                           0.97     28481
   macro avg       0.53      0.95      0.55     28481
weighted avg       1.00      0.97      0.99     28481



Probamos otros clasificadores débiles como LogisticRegression o SVM.

## LogisticRegression

In [42]:
X = df_reduce_RFC_instances_hard.drop(columns=['Class'])
y = df_reduce_RFC_instances_hard['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("Dimensiones de los conjuntos:")
print(f"Conjunto de entrenamiento: {X_train.shape}, {y_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}, {y_test.shape}")

base_estimator=LogisticRegression(max_iter=1000)  
adaboost = AdaBoostClassifier(estimator=base_estimator, random_state=0)


param_grid = {
    'n_estimators': [10, 50, 100],      # Número de clasificadores débiles
    'learning_rate': [0.1, 0.5, 1.0],  # Tasa de aprendizaje
}

# Creamos el GridSearchCV
grid_search = GridSearchCV(estimator=adaboost, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Mostramos los mejores parámetros y el desempeño
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

# Evaluamos el mejor modelo en los datos de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Informe completo en datos de prueba\n", classification_report(y_test, y_pred))

Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Mejores parámetros: {'learning_rate': 0.5, 'n_estimators': 10}
Mejor puntuación de validación cruzada: 0.9223454200379582
Informe completo en datos de prueba
               precision    recall  f1-score   support

           0       0.87      0.96      0.91        89
           1       0.95      0.85      0.90        89

    accuracy                           0.90       178
   macro avg       0.91      0.90      0.90       178
weighted avg       0.91      0.90      0.90       178



### Aplicamos adaBoost con los mejores parámetros en el conjunto de test

In [43]:
# Configura el modelo con los mejores parámetros
base_estimator = LogisticRegression(max_iter=1000)   
adaboost = AdaBoostClassifier(
    estimator=base_estimator,
    n_estimators=10,         
    learning_rate=0.5,       
    random_state=0           
)


X = df_reduce_RFC_instances_hard.drop(columns=['Class'])
y = df_reduce_RFC_instances_hard['Class']

# Entrenamos el modelo con los datos de entrenamiento
adaboost.fit(X, y)

# Evalúa el modelo en los datos de prueba
y_pred_lr = adaboost.predict(X_reduce_RFC)

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test_final, y_pred_lr)
report = classification_report(y_test_final, y_pred_lr, target_names=['Correctas', 'Fraudulentas'])
# Mostrar la matriz de confusión
print("Matriz de confusión:")
print(conf_matrix)

print("Reporte de Clasificación:")
print(report)

Matriz de confusión:
[[28070   362]
 [    4    45]]
Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.99      0.99     28432
Fraudulentas       0.11      0.92      0.20        49

    accuracy                           0.99     28481
   macro avg       0.56      0.95      0.60     28481
weighted avg       1.00      0.99      0.99     28481



## SVM

In [44]:
X = df_reduce_RFC_instances_hard.drop(columns=['Class'])
y = df_reduce_RFC_instances_hard['Class']
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("Dimensiones de los conjuntos:")
print(f"Conjunto de entrenamiento: {X_train.shape}, {y_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}, {y_test.shape}")

base_estimator = SVC(kernel='linear', random_state=0) 
adaboost = AdaBoostClassifier(estimator=base_estimator, random_state=0)


param_grid = {
    'n_estimators': [10, 50, 100],      # Número de clasificadores débiles
    'learning_rate': [0.1, 0.5, 1.0],  # Tasa de aprendizaje
    'estimator__C': [0.1, 1, 10]
}

# Creamos el GridSearchCV
grid_search = GridSearchCV(estimator=adaboost, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Mostramos los mejores parámetros y el desempeño
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

# Evaluamos el mejor modelo en los datos de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Informe completo en datos de prueba\n", classification_report(y_test, y_pred))

Dimensiones de los conjuntos:
Conjunto de entrenamiento: (708, 10), (708,)
Conjunto de prueba: (178, 10), (178,)
Mejores parámetros: {'estimator__C': 1, 'learning_rate': 0.5, 'n_estimators': 10}
Mejor puntuación de validación cruzada: 0.9237338927180101
Informe completo en datos de prueba
               precision    recall  f1-score   support

           0       0.85      0.96      0.90        89
           1       0.95      0.83      0.89        89

    accuracy                           0.89       178
   macro avg       0.90      0.89      0.89       178
weighted avg       0.90      0.89      0.89       178



### Aplicamos adaBoost con los mejores parámetros en el conjunto de test

In [45]:
# Configura el modelo con los mejores parámetros
base_estimator = SVC(kernel='linear', random_state=0, C=1)   
adaboost = AdaBoostClassifier(
    estimator=base_estimator,
    n_estimators=10,         
    learning_rate=0.5,       
    random_state=0           
)


X = df_reduce_RFC_instances_hard.drop(columns=['Class'])
y = df_reduce_RFC_instances_hard['Class']

# Entrenamos el modelo con los datos de entrenamiento
adaboost.fit(X, y)

# Evalúa el modelo en los datos de prueba
y_pred_lr = adaboost.predict(X_reduce_RFC)

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test_final, y_pred_lr)
report = classification_report(y_test_final, y_pred_lr, target_names=['Correctas', 'Fraudulentas'])
# Mostrar la matriz de confusión
print("Matriz de confusión:")
print(conf_matrix)

print("Reporte de Clasificación:")
print(report)

Matriz de confusión:
[[28186   246]
 [    5    44]]
Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.99      1.00     28432
Fraudulentas       0.15      0.90      0.26        49

    accuracy                           0.99     28481
   macro avg       0.58      0.94      0.63     28481
weighted avg       1.00      0.99      0.99     28481

